In [1]:
import os
import torch
import torchvision
from torchvision import datasets
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt
import requests
from zipfile import ZipFile
from io import BytesIO
import numpy as np
import zipfile
import os


zip_file_path = r'C:\Users\nicol\Documents\PoliTo\AdvancedML\project\SPair-71k.zip' 
extract_dir = r'C:\Users\nicol\Documents\PoliTo\AdvancedML\project\SPair-71k_extracted'

# Crea la directory di estrazione se non esiste
os.makedirs(extract_dir, exist_ok=True)

# Estrai il file ZIP solo se esiste
if os.path.exists(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"File '{zip_file_path}' estratto con successo nella directory '{extract_dir}'")
    print(f"Contenuti della directory '{extract_dir}':\n{os.listdir(extract_dir)}")
else:
    print(f"File zip '{zip_file_path}' non trovato. Assicurati che il dataset sia estratto in '{extract_dir}'.")



File 'C:\Users\nicol\Documents\PoliTo\AdvancedML\project\SPair-71k.zip' estratto con successo nella directory 'C:\Users\nicol\Documents\PoliTo\AdvancedML\project\SPair-71k_extracted'
Contenuti della directory 'C:\Users\nicol\Documents\PoliTo\AdvancedML\project\SPair-71k_extracted':
['SPair-71k']


In [2]:
from PIL import Image
import glob
import json


class Normalize(object):
    def __init__(self, image_keys):
        self.image_keys = image_keys
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    def __call__(self, image):
        for key in self.image_keys:
            image[key] /= 255.0
            image[key] = self.normalize(image[key])
        return image


def read_img(path):
    img = np.array(Image.open(path).convert('RGB'))

    return torch.tensor(img.transpose(2, 0, 1).astype(np.float32))


class SPairDataset(Dataset):
    def __init__(self, pair_ann_path, layout_path, image_path, dataset_size, pck_alpha, datatype):

        self.datatype = datatype
        self.pck_alpha = pck_alpha
        self.ann_files = open(os.path.join(layout_path, dataset_size, datatype + '.txt'), "r").read().split('\n')
        self.ann_files = self.ann_files[:len(self.ann_files) - 1]
        self.pair_ann_path = pair_ann_path
        self.image_path = image_path
        self.categories = list(map(lambda x: os.path.basename(x), glob.glob('%s/*' % image_path)))
        self.categories.sort()
        self.transform = Normalize(['src_img', 'trg_img'])

    def __len__(self):
        return len(self.ann_files)

    def __getitem__(self, idx):
        
        raw_line = self.ann_files[idx]
        ann_filename = raw_line.replace(':', '_')
        ann_file = ann_filename + '.json'
        json_path = os.path.join(self.pair_ann_path, self.datatype, ann_file)

        with open(json_path) as f:
            annotation = json.load(f)

        category = annotation['category']
        src_img = read_img(os.path.join(self.image_path, category, annotation['src_imname']))
        trg_img = read_img(os.path.join(self.image_path, category, annotation['trg_imname']))

        trg_bbox = annotation['trg_bndbox']
        pck_threshold = max(trg_bbox[2] - trg_bbox[0],  trg_bbox[3] - trg_bbox[1]) * self.pck_alpha

        sample = {'pair_id': annotation['pair_id'],
                  'filename': annotation['filename'],
                  'src_imname': annotation['src_imname'],
                  'trg_imname': annotation['trg_imname'],
                  'src_imsize': src_img.size(),
                  'trg_imsize': trg_img.size(),

                  'src_bbox': annotation['src_bndbox'],
                  'trg_bbox': annotation['trg_bndbox'],
                  'category': annotation['category'],

                  'src_pose': annotation['src_pose'],
                  'trg_pose': annotation['trg_pose'],

                  'src_img': src_img,
                  'trg_img': trg_img,
                  'src_kps': torch.tensor(annotation['src_kps']).float(),
                  'trg_kps': torch.tensor(annotation['trg_kps']).float(),

                  'mirror': annotation['mirror'],
                  'vp_var': annotation['viewpoint_variation'],
                  'sc_var': annotation['scale_variation'],
                  'truncn': annotation['truncation'],
                  'occlsn': annotation['occlusion'],

                  'pck_threshold': pck_threshold}

        if self.transform:
            sample = self.transform(sample)

        return sample

if __name__ == '__main__':
    base_dir = r"C:\Users\nicol\Documents\PoliTo\AdvancedML\project\SPair-71k_extracted\SPair-71k\SPair-71k"    
    pair_ann_path = os.path.join(base_dir, 'PairAnnotation')
    layout_path = os.path.join(base_dir, 'Layout')
    image_path = os.path.join(base_dir, 'JPEGImages')
    dataset_size = 'large'
    pck_alpha = 0.05
    
    # Verifica che i percorsi esistano prima di creare il dataset
    if os.path.exists(pair_ann_path) and os.path.exists(layout_path) and os.path.exists(image_path):
        trn_dataset = SPairDataset(pair_ann_path, layout_path, image_path, dataset_size, pck_alpha, datatype='trn')
        val_dataset = SPairDataset(pair_ann_path, layout_path, image_path, dataset_size, pck_alpha, datatype='val')
        test_dataset = SPairDataset(pair_ann_path, layout_path, image_path, dataset_size, pck_alpha, datatype='test')

        trn_dataloader = DataLoader(trn_dataset, num_workers=0)
        val_dataloader = DataLoader(val_dataset, num_workers=0)
        test_dataloader = DataLoader(test_dataset, num_workers=0)
        print("Dataset caricati correttamente.")
    else:
        print(f"Errore: Impossibile trovare i percorsi del dataset in '{base_dir}'.\nVerifica l'estrazione e controlla se la struttura delle cartelle corrisponde.")

Dataset caricati correttamente.


In [3]:
# PCK PER POINT - OFFICIAL DINOv2 VERSION (INTERMEDIATE LAYERS + PADDING FIX)
import torch
import math 
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm

# 1. LOAD OFFICIAL MODEL
print("Loading Official DINOv2 Model from Torch Hub...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')
model.to(device)
model.eval() 

print(f"Model loaded on: {device}")

# Helper function for Padding
def pad_to_multiple(x, k=14):
    """
    Pads the image (bottom and right) so that H and W are multiples of k.
    """
    h, w = x.shape[-2:]
    new_h = math.ceil(h / k) * k
    new_w = math.ceil(w / k) * k
    
    pad_bottom = new_h - h
    pad_right = new_w - w
    
    if pad_bottom == 0 and pad_right == 0:
        return x
    return F.pad(x, (0, pad_right, 0, pad_bottom), value=0)

# Initialize counters
total_keypoints = 0
correct_kps_0_05 = 0
class_pck_data = {}

with torch.no_grad(): # Disable gradients
    for i, data in enumerate(tqdm(test_dataloader, desc="Evaluation")):
        
        category = data['category'][0]
        if category not in class_pck_data:
            class_pck_data[category] = {
                'total_keypoints': 0,
                'correct_kps_0_05': 0
            }

        src_img = data['src_img'].to(device)
        trg_img = data['trg_img'].to(device)
        

        # --- FIX: APPLY PADDING ---
        # Ensure dimensions are multiples of 14 to avoid AssertionError
        src_img_padded = pad_to_multiple(src_img, 14)
        trg_img_padded = pad_to_multiple(trg_img, 14)

        # 2. FORWARD PASS (INTERMEDIATE LAYERS STRATEGY)
        # Instead of just the last layer, we grab the last 4 layers.
        # reshape=False -> Returns [Batch, Patch, Channels]
        # return_class_token=False -> Automatically removes CLS token!
        
        n_layers = 4
        layers_src = model.get_intermediate_layers(src_img_padded, n=n_layers, reshape=False, return_class_token=False)
        layers_trg = model.get_intermediate_layers(trg_img_padded, n=n_layers, reshape=False, return_class_token=False)
        
        # Concatenate the layers along the feature dimension (dim=-1)
        # Each layer has 768 dim. 4 layers -> 768 * 4 = 3072 dim.
        feats_src = torch.cat(layers_src, dim=-1) 
        feats_trg = torch.cat(layers_trg, dim=-1)
        
        # --- IMPORTANT: GRID CALCULATION ---
        # We must use PADDED dimensions for the grid
        _, _, H_padded, W_padded = src_img_padded.shape 
        
        # We keep ORIGINAL dimensions for valid boundary checks
        _, _, H_orig, W_orig = data['src_img'].shape
        print(f"H_origin {H_orig}, W_origin{W_orig}\n ")

        patch_size = 14
        w_grid = W_padded // patch_size 
        h_grid = H_padded // patch_size

        kps_list_src = data['src_kps'][0] 
        trg_kps_gt = data['trg_kps'][0] 
        
        # Get threshold value
        pck_threshold = data['pck_threshold'][0].item() 
        
        for n_keypoint, keypoint_src in enumerate(kps_list_src):

            x_src_val = keypoint_src[0].item()
            y_src_val = keypoint_src[1].item()

            # NaN Check
            if math.isnan(x_src_val) or math.isnan(y_src_val):
                continue
            
            x_pixel_src = int(x_src_val)
            y_pixel_src = int(y_src_val)

            # Boundary Check on ORIGINAL image
            if not (0 <= x_pixel_src < W_orig and 0 <= y_pixel_src < H_orig):
                continue

            # Grid Clamp
            x_patch_src = min(x_pixel_src // patch_size, w_grid - 1)
            y_patch_src = min(y_pixel_src // patch_size, h_grid - 1)

            # 3. INDEX CALCULATION
            # No +1 needed because get_intermediate_layers(return_class_token=False) removes it
            patch_index_src = (y_patch_src * w_grid) + x_patch_src

            # Safety clamp for index
            if patch_index_src >= feats_src.shape[1]:
                patch_index_src = feats_src.shape[1] - 1

            # Extract Vector (Now it is size 3072 instead of 768)
            source_vec = feats_src[0, patch_index_src, :]

            # Cosine Similarity
            similarity_map = torch.cosine_similarity(source_vec, feats_trg[0], dim=-1)
            
            # Prediction
            patch_idx_spatial = torch.argmax(similarity_map).item()

            # Convert Index -> Grid -> Pixel
            x_col_pred = patch_idx_spatial % w_grid
            y_row_pred = patch_idx_spatial // w_grid

            x_pred_pixel = x_col_pred * patch_size + (patch_size // 2)
            y_pred_pixel = y_row_pred * patch_size + (patch_size // 2)

            # Ground Truth Check
            gt_x = trg_kps_gt[n_keypoint, 0].item()
            gt_y = trg_kps_gt[n_keypoint, 1].item()

            if math.isnan(gt_x) or math.isnan(gt_y):
                continue
            if not (0 <= gt_x < W_orig and 0 <= gt_y < H_orig):
                continue

            # Distance & Update
            distance = math.sqrt((x_pred_pixel - gt_x)**2 + (y_pred_pixel - gt_y)**2)

            class_pck_data[category]['total_keypoints'] += 1
            if distance <= pck_threshold:
                class_pck_data[category]['correct_kps_0_05'] += 1
        
        # Free memory (Important with bigger feature vectors)
        del feats_src, feats_trg, layers_src, layers_trg, src_img_padded, trg_img_padded
        torch.cuda.empty_cache()

Loading Official DINOv2 Model from Torch Hub...


Using cache found in C:\Users\nicol/.cache\torch\hub\facebookresearch_dinov2_main
C:\Users\nicol/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\nicol/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\nicol/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


Model loaded on: cuda


Evaluation:   0%|          | 1/12234 [00:00<2:31:05,  1.35it/s]

H_origin 333, W_origin500
 
H_origin 333, W_origin500
 


...

H_origin 500, W_origin375
 
H_origin 500, W_origin375
 


Evaluation: 100%|█████████▉| 12232/12234 [27:42<00:00,  7.09it/s]

H_origin 500, W_origin375
 
H_origin 500, W_origin375
 


Evaluation: 100%|██████████| 12234/12234 [27:42<00:00,  7.36it/s]

H_origin 500, W_origin375
 


In [5]:
# CALCOLO PCK PER IMAGE (OFFICIAL DINOv2 VERSION + PADDING FIX)
import torch
import math 
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm

# 1. LOAD OFFICIAL MODEL (Replaces Hugging Face)
print("Loading Official DINOv2 Model from Torch Hub...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')
model.to(device)
model.eval()

print(f"Modello caricato su: {device}")

# Helper function for Padding (Crucial for DINOv2 official)
def pad_to_multiple(x, k=14):
    """
    Pads the image (bottom and right) so that H and W are multiples of k.
    """
    h, w = x.shape[-2:]
    new_h = math.ceil(h / k) * k
    new_w = math.ceil(w / k) * k
    
    pad_bottom = new_h - h
    pad_right = new_w - w
    
    if pad_bottom == 0 and pad_right == 0:
        return x
    return F.pad(x, (0, pad_right, 0, pad_bottom), value=0)

# Inizializza i contatori per la metrica PCK
class_pck_data = {}

with torch.no_grad(): # Disabilita il calcolo dei gradienti
    for i, data in enumerate(tqdm(test_dataloader, desc="Valutazione")):

        # Retrieve the category for the current item
        category = data['category'][0]

        # Initialize category entry in class_pck_data if it doesn't exist
        if category not in class_pck_data:
            class_pck_data[category] = {
                'total_image': 0,
                'image_value_sum': 0, # Accumulatore per le medie delle singole immagini
            }

        # Counters specific for THIS image
        img_tot_keypoints = 0
        img_correct_keypoints = 0

        src_img = data['src_img'].to(device)
        trg_img = data['trg_img'].to(device)

        # --- FIX: APPLY PADDING ---
        # Ensure dimensions are multiples of 14 to avoid AssertionError
        src_img_padded = pad_to_multiple(src_img, 14)
        trg_img_padded = pad_to_multiple(trg_img, 14)

        n_layers = 4
        layers_src = model.get_intermediate_layers(src_img_padded, n=n_layers, reshape=False, return_class_token=False)
        layers_trg = model.get_intermediate_layers(trg_img_padded, n=n_layers, reshape=False, return_class_token=False)
        
        # Concatenate the layers along the feature dimension (dim=-1)
        # Each layer has 768 dim. 4 layers -> 768 * 4 = 3072 dim.
        feats_src = torch.cat(layers_src, dim=-1) 
        feats_trg = torch.cat(layers_trg, dim=-1)

       
        # --- IMPORTANT: GRID CALCULATION ---
        # We must use PADDED dimensions for the grid, otherwise indices will drift
        _, _, H_padded, W_padded = src_img_padded.shape 
        
        # We keep ORIGINAL dimensions for valid boundary checks (Ground Truth)
        _, _, H_orig, W_orig = data['src_img'].shape

        patch_size = 14
        w_grid = W_padded // patch_size 
        h_grid = H_padded // patch_size

        kps_list_src = data['src_kps'][0] 
        trg_kps_gt = data['trg_kps'][0] 
        
        # Estrai threshold value dal tensore
        pck_threshold = data['pck_threshold'][0].item() 
        
        for n_keypoint, keypoint_src in enumerate(kps_list_src):

            x_src_val = keypoint_src[0].item()
            y_src_val = keypoint_src[1].item()

            # CHECK 1: NaN / Validità Keypoint Sorgente
            if math.isnan(x_src_val) or math.isnan(y_src_val):
                continue
            
            x_pixel_src = int(x_src_val)
            y_pixel_src = int(y_src_val)

            # Boundary Check on ORIGINAL image (ignore points in padded area)
            if not (0 <= x_pixel_src < W_orig and 0 <= y_pixel_src < H_orig):
                continue

            # CHECK 2: Grid Clamp
            x_patch_src = min(x_pixel_src // patch_size, w_grid - 1)
            y_patch_src = min(y_pixel_src // patch_size, h_grid - 1)

            # CALCOLO INDICE (OFFICIAL LOGIC)
            # feats_src NON ha il CLS token. Non serve aggiungere +1.
            patch_index_src = (y_patch_src * w_grid) + x_patch_src

            # Safety check per l'indice
            if patch_index_src >= feats_src.shape[1]:
                patch_index_src = feats_src.shape[1] - 1

            # Estrai feature sorgente
            source_vec = feats_src[0, patch_index_src, :]

            # Similarità con tutte le patch target
            similarity_map = torch.cosine_similarity(source_vec, feats_trg[0], dim=-1)
            
            # L'argmax ci da l'indice spaziale diretto
            patch_idx_spatial = torch.argmax(similarity_map).item()

            # Conversione indice -> coordinate griglia
            x_col_pred = patch_idx_spatial % w_grid
            y_row_pred = patch_idx_spatial // w_grid

            # Conversione griglia -> pixel (centro della patch)
            x_pred_pixel = x_col_pred * patch_size + (patch_size // 2)
            y_pred_pixel = y_row_pred * patch_size + (patch_size // 2)

            gt_x = trg_kps_gt[n_keypoint, 0].item()
            gt_y = trg_kps_gt[n_keypoint, 1].item()

            # CHECK 3: Validità Keypoint Target (Ground Truth)
            if math.isnan(gt_x) or math.isnan(gt_y):
                continue
            if not (0 <= gt_x < W_orig and 0 <= gt_y < H_orig):
                continue

            # Calcola distanza
            distance = math.sqrt((x_pred_pixel - gt_x)**2 + (y_pred_pixel - gt_y)**2)

            # Aggiorna contatori per QUESTA immagine
            img_tot_keypoints += 1
            if distance <= pck_threshold:
                img_correct_keypoints += 1
        
        # AGGIORNAMENTO DATI CATEGORIA (PCK PER IMAGE)
        # Se l'immagine aveva almeno un punto valido, calcoliamo la sua accuratezza
        if img_tot_keypoints > 0:
            image_accuracy = img_correct_keypoints / img_tot_keypoints
            
            class_pck_data[category]['total_image'] += 1
            class_pck_data[category]['image_value_sum'] += image_accuracy
        
        

Loading Official DINOv2 Model from Torch Hub...


Using cache found in C:\Users\nicol/.cache\torch\hub\facebookresearch_dinov2_main


Modello caricato su: cuda


Valutazione: 100%|██████████| 12234/12234 [48:44<00:00,  4.18it/s]     


In [6]:
# CALCOLO PCK PER IMAGE
print("--- PCK per Class ---")
class_pck_0_05_list = []
#class_pck_0_1_list = []
#class_pck_0_2_list = []

for category, data in class_pck_data.items():
    total_image = data['total_image']
    correct_image_0_05 = data['image_value_sum']
    #correct_image_0_1 = data['image_value_sum']
    #correct_image_0_2 = data['image_value_sum']

    pck_0_05 = (correct_image_0_05 / total_image) * 100 if total_image > 0 else 0
    #pck_0_1 = (correct_image_0_1 / total_image) * 100 if total_image > 0 else 0
    #pck_0_2 = (correct_image_0_2 / total_image) * 100 if total_image > 0 else 0

    print(f"Category: {category}")
    print(f"  PCK@0.05: {pck_0_05:.2f}% ({correct_image_0_05}/{total_image})")
    #print(f"  PCK@0.1: {pck_0_1:.2f}% ({correct_image_0_1}/{total_image})")
    #print(f"  PCK@0.2: {pck_0_2:.2f}% ({correct_image_0_2}/{total_image})")
    print("-" * 20)

    if total_image> 0: # Only add to the list if there were keypoints for this class
        class_pck_0_05_list.append(pck_0_05)
        #class_pck_0_1_list.append(pck_0_1)
        #class_pck_0_2_list.append(pck_0_2)

# 4. Calculate and Display Overall Mean PCK
print("\n--- Overall Mean PCK ---")
overall_mean_pck_0_05 = sum(class_pck_0_05_list) / len(class_pck_0_05_list) if class_pck_0_05_list else 0
#overall_mean_pck_0_1 = sum(class_pck_0_1_list) / len(class_pck_0_1_list) if class_pck_0_1_list else 0
#overall_mean_pck_0_2 = sum(class_pck_0_2_list) / len(class_pck_0_2_list) if class_pck_0_2_list else 0

print(f"Overall Mean PCK@0.05: {overall_mean_pck_0_05:.2f}%")
#print(f"Overall Mean PCK@0.1: {overall_mean_pck_0_1:.2f}%")
#print(f"Overall Mean PCK@0.2: {overall_mean_pck_0_2:.2f}%")



--- PCK per Class ---
Category: aeroplane
  PCK@0.05: 22.18% (152.40139104101974/687)
--------------------
Category: bicycle
  PCK@0.05: 9.48% (59.70865800865805/630)
--------------------
Category: bird
  PCK@0.05: 23.14% (162.4627927627926/702)
--------------------
Category: boat
  PCK@0.05: 6.47% (42.07698412698412/650)
--------------------
Category: bottle
  PCK@0.05: 5.03% (42.326190476190504/841)
--------------------
Category: bus
  PCK@0.05: 13.31% (84.79045464339585/637)
--------------------
Category: car
  PCK@0.05: 10.59% (58.75425685425689/555)
--------------------
Category: cat
  PCK@0.05: 36.03% (215.83100788100788/599)
--------------------
Category: chair
  PCK@0.05: 3.33% (20.664971139971133/621)
--------------------
Category: cow
  PCK@0.05: 14.35% (91.3805957802089/637)
--------------------
Category: dog
  PCK@0.05: 17.97% (107.82708680208684/600)
--------------------
Category: horse
  PCK@0.05: 4.57% (26.983311133311123/590)
--------------------
Category: motorbike
  P

In [4]:
#PCK per point
print("--- PCK per Class ---")
class_pck_0_05_list = []
#class_pck_0_1_list = []
#class_pck_0_2_list = []

for category, data in class_pck_data.items():
    total_kps = data['total_keypoints']
    correct_kps_0_05 = data['correct_kps_0_05']
    #correct_kps_0_1 = data['correct_kps_0_1']
    #correct_kps_0_2 = data['correct_kps_0_2']

    pck_0_05 = (correct_kps_0_05 / total_kps) * 100 if total_kps > 0 else 0
    #pck_0_1 = (correct_kps_0_1 / total_kps) * 100 if total_kps > 0 else 0
    #pck_0_2 = (correct_kps_0_2 / total_kps) * 100 if total_kps > 0 else 0

    print(f"Category: {category}")
    print(f"  PCK@0.05: {pck_0_05:.2f}% ({correct_kps_0_05}/{total_kps})")
    #print(f"  PCK@0.1: {pck_0_1:.2f}% ({correct_kps_0_1}/{total_kps})")
    #print(f"  PCK@0.2: {pck_0_2:.2f}% ({correct_kps_0_2}/{total_kps})")
    print("-" * 20)

    if total_kps > 0: # Only add to the list if there were keypoints for this class
        class_pck_0_05_list.append(pck_0_05)
        #class_pck_0_1_list.append(pck_0_1)
        #class_pck_0_2_list.append(pck_0_2)

# 4. Calculate and Display Overall Mean PCK
print("\n--- Overall Mean PCK ---")
overall_mean_pck_0_05 = sum(class_pck_0_05_list) / len(class_pck_0_05_list) if class_pck_0_05_list else 0
#overall_mean_pck_0_1 = sum(class_pck_0_1_list) / len(class_pck_0_1_list) if class_pck_0_1_list else 0
#overall_mean_pck_0_2 = sum(class_pck_0_2_list) / len(class_pck_0_2_list) if class_pck_0_2_list else 0

print(f"Overall Mean PCK@0.05: {overall_mean_pck_0_05:.2f}%")
#print(f"Overall Mean PCK@0.1: {overall_mean_pck_0_1:.2f}%")
#print(f"Overall Mean PCK@0.2: {overall_mean_pck_0_2:.2f}%")



--- PCK per Class ---
Category: aeroplane
  PCK@0.05: 25.85% (1349/5218)
--------------------
Category: bicycle
  PCK@0.05: 11.78% (414/3513)
--------------------
Category: bird
  PCK@0.05: 25.70% (1021/3972)
--------------------
Category: boat
  PCK@0.05: 8.06% (243/3016)
--------------------
Category: bottle
  PCK@0.05: 6.10% (330/5409)
--------------------
Category: bus
  PCK@0.05: 19.51% (815/4178)
--------------------
Category: car
  PCK@0.05: 15.57% (522/3352)
--------------------
Category: cat
  PCK@0.05: 37.73% (2215/5870)
--------------------
Category: chair
  PCK@0.05: 4.38% (130/2968)
--------------------
Category: cow
  PCK@0.05: 16.44% (823/5007)
--------------------
Category: dog
  PCK@0.05: 20.54% (928/4517)
--------------------
Category: horse
  PCK@0.05: 5.59% (227/4064)
--------------------
Category: motorbike
  PCK@0.05: 11.16% (347/3110)
--------------------
Category: person
  PCK@0.05: 13.40% (544/4061)
--------------------
Category: pottedplant
  PCK@0.05: 4.21% (